In [1]:
import jax
import flax
import hard_not
import harden

In [4]:
def hcall(f, x):
    soft, hard, symbolic = x
    soft = f(soft)(x)
    return soft

In [28]:
soft, hard, symbolic = hard_not.NotLayer(layer_size=4)
soft_weights = soft.init(jax.random.PRNGKey(0), [1.0, 1.0])
x = soft.apply(soft_weights, [1.0, 0.0])
x = jax.numpy.ravel(x)
hard_weights = harden.harden(soft_weights)
y = hard.apply(hard_weights, [True, False])
y = jax.numpy.ravel(y)
z = hard.apply(hard_weights, ['True', 'False'])
z = jax.numpy.ravel(z)
print(x, hard, z)

TypeError: Value 'True' with dtype <U4 is not a valid JAX array type. Only arrays of numeric types are supported by JAX.

In [4]:
soft_net, hard_net, symbolic_net = hard_not.NotLayer(layer_size=4)
soft_weights = soft_net.init(jax.random.PRNGKey(0), [1.0, 1.0])
hard_weights = harden.harden_dict(soft_weights)
[
    soft_net.apply(soft_weights, [1.0, 1.0]),
    hard_net.apply(hard_weights, [True, True]),
    symbolic_net.apply(hard_weights, ['x1', 'x2']),
    eval(symbolic_net.apply(hard_weights, [True, True])),
]

[DeviceArray([[0.26377404, 0.8707025 ],
              [0.44444847, 0.03216302],
              [0.6110164 , 0.685097  ],
              [0.9133855 , 0.08662593]], dtype=float32),
 DeviceArray([[False,  True],
              [False, False],
              [ True,  True],
              [ True, False]], dtype=bool),
 '[[not(x1 ^ False), not(x2 ^ True)], [not(x1 ^ False), not(x2 ^ False)], [not(x1 ^ True), not(x2 ^ True)], [not(x1 ^ True), not(x2 ^ False)]]',
 [[False, True], [False, False], [True, True], [True, False]]]